# AT&T Spam Detector - BLOC 5
## Final : Building the Spam Detector
Developed by Myriam Goyet     
Contact : https://www.linkedin.com/in/myriamgoyet/

# 1. Spam detector with Baseline model

Although there is room for performance optimization, our Baseline model is already ready for testing as a spam detector

To test our model, let's apply the same pre-processing to a new randomly selected message:    
We use the model and the tokenizer already trained on train set.       

In [16]:
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.lang.en.stop_words import STOP_WORDS
import tensorflow as tf
import joblib
import os
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import json

def predict_message(model, text, tokenizer, max_len):
    """
    Applies preprocessing, tokenization, padding, and prediction to a single message.

    Parameters:
    model: The trained model for prediction.
    text (str): The text message to preprocess and predict.
    tokenizer: The tokenizer fitted on the full vocabulary.
    max_len (int): The maximum length of the sequences.

    Returns:
    str: The prediction result, either "Spam" or "Ham".
    """
    # Step 1: Clean the text
    text = ''.join(ch for ch in text if ch.isalnum() or ch == " " or ch == "'")
    text = ' '.join(text.split())
    text = text.lower()
    text = text.strip()

    # Step 2: Lemmatize and remove stopwords
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in STOP_WORDS and token.text not in STOP_WORDS]
    preprocessed_text = " ".join(tokens)

    # Step 3: Tokenization
    sequence = tokenizer.texts_to_sequences([preprocessed_text])

    # Step 4: Padding
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len, padding='post')

    # Step 5: Prediction
    pred = model.predict(padded_sequence)[0][0]
    return "Spam" if pred > 0.5 else "Ham"


2025-07-09 00:18:20.939225: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-09 00:18:21.057697: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-09 00:18:21.559512: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-09 00:18:23.095640: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# Exemple of use on a randomly selected message:
import pandas as pd


df= pd.read_csv("AT&T_data_preprocessed.csv")
model = joblib.load("Models_trained/Baseline_model.joblib")
with open("Models_trained/tokenizer_baseline_model.json") as f:
    tokenizer = tokenizer_from_json(f.read())
max_len = 100

text = df["messages"][1]
target = df["target"][1]
prediction = predict_message(model, text, tokenizer, max_len)
print('Text :',text)
print('Target :',target)

print("The model predicts that the message is a",prediction)
if prediction == "Spam" and target == 1:
  print("Prediction correct !")
elif prediction == "Ham" and target == 0:
  print("Prediction correct !")
else:
  print("Prediction wrong !")


1/1 [==============================] - 0s 211ms/step
Text : Ok lar... Joking wif u oni...
Target : 0
The model predicts that the message is a Ham
Prediction correct !


In [18]:
# Example 2 with spam
text = df["messages"][5]
target = df["target"][5]
prediction = predict_message(model, text, tokenizer, max_len)
print('Text :',text)
print('Target :',target)

print("The model predicts that the message is a",prediction)
if prediction == "Spam" and target == 1:
  print("Prediction correct !")
elif prediction == "Ham" and target == 0:
  print("Prediction correct !")
else:
  print("Prediction wrong !")

1/1 [==============================] - 0s 17ms/step
Text : FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv
Target : 1
The model predicts that the message is a Spam
Prediction correct !


In [19]:
# Exemple 3 with new unseen message
text = "Congratulations! You've won a £500 gift card. Click here to claim: http://winbigprizes.co.uk. Reply STOP to opt-out."
target = 1
prediction = predict_message(model, text, tokenizer, max_len)
print('Text :',text)
print('Target :',target)

print("The model predicts that the message is a",prediction)
if prediction == "Spam" and target == 1:
  print("Prediction correct !")
elif prediction == "Ham" and target == 0:
  print("Prediction correct !")
else:
  print("Prediction wrong !")

1/1 [==============================] - 0s 17ms/step
Text : Congratulations! You've won a £500 gift card. Click here to claim: http://winbigprizes.co.uk. Reply STOP to opt-out.
Target : 1
The model predicts that the message is a Spam
Prediction correct !


In [20]:
# Exemple 4 with new unseen message
text = "Congratulation for your Master degree ! Proud of you !."
target = 0
prediction = predict_message(model, text, tokenizer, max_len)
print('Text :',text)
print('Target :',target)

print("The model predicts that the message is a",prediction)
if prediction == "Spam" and target == 1:
  print("Prediction correct !")
elif prediction == "Ham" and target == 0:
  print("Prediction correct !")
else:
  print("Prediction wrong !")

1/1 [==============================] - 0s 18ms/step
Text : Congratulation for your Master degree ! Proud of you !.
Target : 0
The model predicts that the message is a Ham
Prediction correct !


# 2. Spam detector with Best DistilBERT + Logistic Regression model

This model works the same way, but instead of having only a model and a tokenizer to load, we also have a classifier:

In [21]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
import numpy as np
import joblib
import json
import os
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS

# Load SpaCy once
nlp = en_core_web_sm.load()

def preprocess_text(text):
    text = ''.join(ch for ch in text if ch.isalnum() or ch == " " or ch == "'")
    text = ' '.join(text.split())
    text = text.lower()
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in STOP_WORDS]
    return " ".join(tokens)

def extract_features(text, tokenizer, model):
    preprocessed = preprocess_text(text)
    
    # Tokenize & get CLS embedding
    inputs = tokenizer(preprocessed, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # shape: (768,)

    # Handcrafted features
    num_words = len(preprocessed.split())
    message_len = len(preprocessed)

    # Final feature vector
    full_features = np.concatenate([cls_embedding, [num_words, message_len]])  # shape: (770,)
    return full_features.reshape(1, -1)  # shape: (1, 770)


In [22]:
# Load all components
distilbert_model = DistilBertModel.from_pretrained("Models_trained/DistilBERT+lr/final_pipeline/distilbert_model")
tokenizer = DistilBertTokenizer.from_pretrained("Models_trained/DistilBERT+lr/final_pipeline/tokenizer")
classifier = joblib.load("Models_trained/DistilBERT+lr/final_pipeline/classifier_pipeline.joblib")


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Pipeline fr

In [23]:
# Load your test message
import pandas as pd
df = pd.read_csv("AT&T_data_preprocessed.csv")
text = df["messages"][10]
target = df["target"][10]

# Extract features and predict
X_new = extract_features(text, tokenizer, distilbert_model)
proba = classifier.predict_proba(X_new)[0][1]
prediction = "Spam" if proba > 0.5 else "Ham"

# Output
print("Text:", text)
print("Target:", "Spam" if target == 1 else "Ham")
print("Prediction:", prediction)

if (prediction == "Spam" and target == 1) or (prediction == "Ham" and target == 0):
    print("✅ Prediction correct!")
else:
    print("❌ Prediction wrong.")


Text: I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.
Target: Ham
Prediction: Ham
✅ Prediction correct!


In [24]:
# Example 2 with spam
text = df["messages"][8]
target = df["target"][8]

# Extract features and predict
X_new = extract_features(text, tokenizer, distilbert_model)
proba = classifier.predict_proba(X_new)[0][1]
prediction = "Spam" if proba > 0.5 else "Ham"

# Output
print("Text:", text)
print("Target:", "Spam" if target == 1 else "Ham")
print("Prediction:", prediction)

if (prediction == "Spam" and target == 1) or (prediction == "Ham" and target == 0):
    print("✅ Prediction correct!")
else:
    print("❌ Prediction wrong.")

Text: WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Target: Spam
Prediction: Spam
✅ Prediction correct!


In [25]:
# Example 3 with new unseen message:
text = "Congratulations! 🎉 You've been selected as the winner of our $1,000 cash prize. Reply 'WIN' now to claim your reward. T&Cs apply."
target = 1

# Extract features and predict
X_new = extract_features(text, tokenizer, distilbert_model)
proba = classifier.predict_proba(X_new)[0][1]
prediction = "Spam" if proba > 0.5 else "Ham"

# Output
print("Text:", text)
print("Target:", "Spam" if target == 1 else "Ham")
print("Prediction:", prediction)

if (prediction == "Spam" and target == 1) or (prediction == "Ham" and target == 0):
    print("✅ Prediction correct!")
else:
    print("❌ Prediction wrong.")

Text: Congratulations! 🎉 You've been selected as the winner of our $1,000 cash prize. Reply 'WIN' now to claim your reward. T&Cs apply.
Target: Spam
Prediction: Spam
✅ Prediction correct!


In [26]:
# Example 4 with new unseen message:
text = "Hey! Just checking in to see if you're still good for lunch tomorrow at 1 PM. Let me know "
target = 0

# Extract features and predict
X_new = extract_features(text, tokenizer, distilbert_model)
proba = classifier.predict_proba(X_new)[0][1]
prediction = "Spam" if proba > 0.5 else "Ham"

# Output
print("Text:", text)
print("Target:", "Spam" if target == 1 else "Ham")
print("Prediction:", prediction)

if (prediction == "Spam" and target == 1) or (prediction == "Ham" and target == 0):
    print("✅ Prediction correct!")
else:
    print("❌ Prediction wrong.")

Text: Hey! Just checking in to see if you're still good for lunch tomorrow at 1 PM. Let me know 
Target: Ham
Prediction: Ham
✅ Prediction correct!
